# Maximización de $f(x) = x\sin(10\pi x) + 1$ en $[0,1]$  

Este notebook contiene un **procedimiento numérico y justificativo** para encontrar el máximo global de la función
\(f(x)=x\sin(10\pi x)+1\) en el intervalo \([0,1]\). Se utiliza:

1. Muestreo en una grilla fina para localizar candidatos.  
2. Detección de máximos locales en la grilla.  
3. Refinamiento de las raíces de la derivada $f'(x)=0$ usando el método de Newton (derivadas analíticas).  
4. Evaluación de candidatos y extremos para seleccionar el máximo global.  

Los bloques de código que siguen implementan exactamente este flujo y muestran la gráfica final.

In [ ]:
import numpy as np
def f(x):
    x = np.asarray(x)
    return x * np.sin(10*np.pi*x) + 1

def f1(x):
    return np.sin(10*np.pi*x) + 10*np.pi*x*np.cos(10*np.pi*x)

def f2(x):
    return 20*np.pi*np.cos(10*np.pi*x) - 100*(np.pi**2)*x*np.sin(10*np.pi*x)

# Muestreo y cálculo en grilla
N = 20001
x_grid = np.linspace(0,1,N)
f_grid = f(x_grid)

# (El resto del flujo sigue: detección de máximos locales, refinamiento con Newton y evaluación final)

## Muestreo para localizar candidatos
Se toma una grilla uniforme en [0,1] y se evalúa la función para localizar máximos locales.

In [ ]:
# Muestreo y detección de máximos locales
idx_max = int(np.argmax(f_grid))
x0 = x_grid[idx_max]
local_max_idx = np.where((f_grid[1:-1] > f_grid[:-2]) & (f_grid[1:-1] > f_grid[2:]))[0] + 1
candidates = list(x_grid[local_max_idx])
if idx_max not in x_grid[local_max_idx]:
    candidates.append(x0)
cand_vals = [(x, float(f(x))) for x in candidates]
cand_vals_sorted = sorted(cand_vals, key=lambda t: t[1], reverse=True)
top_candidates = [t[0] for t in cand_vals_sorted[:10]]
top_candidates


## Refinamiento de las raíces de la derivada
Se usa el método de Newton (derivadas analíticas) para refinar las posiciones de los máximos locales detectados.

In [ ]:
def newton_refine(x0, tol=1e-14, maxiter=200):
    x = float(np.clip(x0, 0.0, 1.0))
    for i in range(maxiter):
        fp = f1(x)
        fpp = f2(x)
        if abs(fpp) < 1e-16:
            break
        dx = fp / fpp
        if abs(dx) > 0.2:
            dx = np.sign(dx)*0.2
        x_new = x - dx
        x_new = min(1.0, max(0.0, x_new))
        if abs(x_new - x) < tol:
            x = x_new
            break
        x = x_new
    return x

roots = []
for xinit in top_candidates:
    xr = newton_refine(xinit)
    xr = float(np.clip(xr, 0.0, 1.0))
    roots.append(xr)

# Deduplicación
roots_unique = []
for r in roots:
    if not any(abs(r - ru) < 1e-10 for ru in roots_unique):
        roots_unique.append(r)
roots_unique = sorted(roots_unique)
roots_unique


## Evaluación final y gráfica
Se evalúan las raíces refinadas y los extremos; se elige el máximo global y se grafica la función con la marca del máximo.

In [ ]:
all_points = roots_unique + [0.0, 1.0]
evals = [(x, float(f(x))) for x in all_points]
x_star, f_star = max(evals, key=lambda t: t[1])
print('Máximo global aproximado: x* = {:.12f}'.format(x_star))
print('f(x*) = {:.12f}'.format(f_star))

import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
plt.plot(x_grid, f_grid)
plt.scatter([x_star], [f_star])
plt.title('f(x) = x·sin(10πx) + 1 — máximo en x* ≈ {:.12f}'.format(x_star))
plt.xlabel('x')
plt.ylabel('f(x)')
plt.grid(True)
plt.show()
